In [0]:
!pip install yfinance

In [0]:
import yfinance as yf
from datetime import datetime,timedelta 
import pandas as pd

In [0]:
spark.catalog.setCurrentDatabase("default")

In [0]:
cnt=spark.sql("select max(Date) from sw_hot_table").first()[0]
#print(cnt)
if cnt is None:
    start=(datetime.now()-timedelta(days=1094)).date()
    end=(datetime.now()+timedelta(days=1)).date()
    print("First")
else: 
   start=spark.sql("Select max(Date) from sw_hot_table").first()[0]
   end=(datetime.now()+timedelta(days=1)).date()
   print("Second")
sw=yf.download("SWARAJENG.NS",start=start,end=end)
sw=sw.reset_index()
sw.columns=[col[0] if isinstance(col,tuple) else col for col in sw.columns]
sw=sw[['Date','Close']].copy()
sw['Date']=sw['Date'].dt.date
sw['Close']=sw['Close'].round(2)
sw['Year'] = pd.to_datetime(sw['Date']).dt.year
sw['Month'] = pd.to_datetime(sw['Date']).dt.month
sw.rename(columns={'Close':'Close_Price'},inplace=True)
sw_=spark.createDataFrame(sw)
sw_.createOrReplaceTempView("new_data")

In [0]:
%sql
merge into sw_hot_table as target
using new_data as source
on source.Date=target.Date
when matched then
update set target.Close_Price=source.Close_Price
when not matched then
insert (Date,Close_Price,year,month) values (source.Date,source.Close_Price,source.Year,source.Month)

In [0]:
%sql
select * from sw_hot_table order by date desc limit 10;

In [0]:
%sql
--delete from sw_hot_table where Close_Price>0;

%sql
--Altering the table for partitoning
alter table sw_hot_table add columns (year int,month int);

update sw_hot_table set year=year(Date);

update sw_hot_table set month=month(Date);


%sql
select * from sw_hot_table;

In [0]:
%sql
select * from sw_hot_table;

%sql
create or replace table sw_hot_table_partitioned
using delta
partitioned by (year,month)
as 
select Date,Close_Price,year(Date) as year,month(Date) as month 
from sw_hot_table

In [0]:
%sql
--create or replace table sw_hot_table_partitioned using delta partitioned by (year,month) as select Date,Close_Price,year(Date) as year,month(Date) as month from sw_hot_table

In [0]:
%sql
select * from sw_hot_table_partitioned

%sql
drop table sw_hot_table;
alter table sw_hot_table_partitioned rename to sw_hot_table;

In [0]:
%sql
select Date 
from sw_hot_table 
order by Date desc 
limit 1;

Creation of Cold Table --To store data older than 3 years/ 36  months..
%sql
create table if not exists sw_cold_table(
  Date date,
  Close_Price decimal(10,2), 
  year int,
  month int
) using delta
partitioned by (year,month);

In [0]:
%sql
describe sw_hot_table